In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_teddynote import logging

logging.langsmith("AngelGate")

LangSmith 추적을 시작합니다.
[프로젝트명]
AngelGate


In [3]:
from typing import TypedDict, List, Dict, Optional


class GraphState(TypedDict):
    # -----------------------------
    # ① 입력 데이터 (앞단 4개 에이전트에서 받아옴)
    # -----------------------------
    startup: Dict[str, any]  # 창업자 및 기업 기본 정보
    technology: Dict[str, any]  # 기술 정보 (핵심 기술, IP, 구현력 등)
    market: Dict[str, any]  # 시장 및 트랙션 데이터
    competition: Dict[str, any]  # 경쟁사 및 진입장벽 관련 정보
    deal_terms: Dict[str, any]  # 밸류에이션 및 투자 조건

    # -----------------------------
    # ② 내부 계산 결과 (투자판단 에이전트가 생성)
    # -----------------------------
    scores: Dict[str, float]  # 카테고리별 점수 (0~5)
    score_details: Dict[str, Dict[str, float]]  # 세부 항목별 점수 (세분화용)
    base_score: float  # 리스크 적용 전 총점 (0~100 기준)
    risk_penalty: float  # 리스크 감점률 (-0.05 ~ -0.15)
    total_score: float  # 리스크 적용 후 총점

    # -----------------------------
    # ③ 판단 결과 및 메타데이터
    # -----------------------------
    decision: str  # 최종 판단 (invest / invest_conditional / watchlist / reject_temp / reject_final)
    decision_reason: str  # 최종 판단에 대한 핵심 요약 (LLM 또는 알고리즘 생성)
    iteration: int  # 현재까지 반복 평가한 횟수 (초기=1)
    tags: List[str]  # 태그 (예: ["watchlist_stage1", "reject_temp"])
    reasons: Dict[str, str]  # 각 점수 항목별 이유 설명 (LLM 또는 하드코딩 해석)

    # -----------------------------
    # ④ 향후 재탐색을 위한 힌트 (보류/거절 시만 사용)
    # -----------------------------
    next_steps: Optional[
        List[str]
    ]  # 다음 루프에서 보완해야 할 지점 (예: ["시장성 데이터 확보", "고객 세그먼트 명확화"])

### Scoring functions

In [4]:
from typing import Dict, Tuple


def score_founder(startup: Dict) -> Tuple[float, Dict[str, float]]:
    """
    Founder & Team 점수를 계산한다. (LLM 보조 포함)

    Parameters:
        startup (dict): 창업자 및 팀 관련 정보. 예: {
            "founder_info": {
                "experience_years": 8,
                "exits": 1,
                "industry_fit": "high",
                "team_composition": "balanced"
            }
        }

    Returns:
        total_score (float): 0~5점 사이 평균 점수
        details (dict): 세부 항목별 점수 {"industry_fit": 4.0, "execution": 3.5, ...}
    """
    # TODO: LLM 기반 평가 + 규칙 기반 스코어링 조합
    total_score = 0.0
    details = {}
    return total_score, details


def score_market(market: Dict) -> Tuple[float, Dict[str, float]]:
    """
    시장성 점수를 계산한다. (하드코딩 중심)

    Parameters:
        market (dict): 시장 정보. 예: {
            "tam_usd": 20000000000,
            "cagr_pct": 35,
            "problem_severity": "high",
            "demand_drivers": ["AI adoption", "automation"]
        }

    Returns:
        total_score (float): 0~5점
        details (dict): 세부 점수 {"tam": 5.0, "cagr": 4.5, "problem": 4.0, "drivers": 4.5}
    """
    total_score = 0.0
    details = {}
    return total_score, details


from typing import Dict, Tuple


def score_technology(technology: Dict) -> Tuple[float, Dict[str, float]]:
    """
    기술력 점수를 계산한다. (70% 하드코딩 + 30% LLM 보조)

    Parameters:
        technology (dict): 기술 정보. 예: {
            "core_technology": "Transformer-based LLM",
            "ip_status": "Patent filed",
            "scalability": "high",
            "performance_gap": {"speed": "+20%", "accuracy": "+5%"}
        }

    Returns:
        total_score (float): 0~5점
        details (dict): {"originality": 4.5, "ip": 4.0, "scalability": 4.0, "performance": 3.5}
    """
    total_score = 0.0
    details = {}
    return total_score, details


def score_moat(competition: Dict) -> Tuple[float, Dict[str, float]]:
    """
    경쟁 우위 점수를 계산한다. (LLM 비중 높음)

    Parameters:
        competition (dict): 경쟁 환경 정보. 예: {
            "entry_barriers": "data moat",
            "network_effects": "strong",
            "ecosystem_position": "emerging",
            "defensibility": "moderate"
        }

    Returns:
        total_score (float): 0~5점
        details (dict): {"entry_barriers": 4.0, "network": 3.5, "ecosystem": 4.0, "defensibility": 3.0}
    """
    total_score = 0.0
    details = {}
    return total_score, details


def score_traction(market: Dict) -> Tuple[float, Dict[str, float]]:
    """
    트랙션 점수를 계산한다. (하드코딩 기반)

    Parameters:
        market (dict): 트랙션 정보 포함. 예: {
            "users": 200000,
            "mom_growth_pct": 18,
            "d30_retention_pct": 35,
            "pmf_signal": "medium"
        }

    Returns:
        total_score (float): 0~5점
        details (dict): {"growth": 4.5, "revenue": 3.5, "retention": 3.0, "pmf": 3.5}
    """
    total_score = 0.0
    details = {}
    return total_score, details


def score_terms(deal_terms: Dict) -> Tuple[float, Dict[str, float]]:
    """
    투자 조건 점수를 계산한다. (하드코딩)

    Parameters:
        deal_terms (dict): 투자 조건. 예: {
            "post_money_usd": 200000000,
            "arr_usd": 25000000,
            "equity_offer_pct": 15
        }

    Returns:
        total_score (float): 0~5점
        details (dict): {"valuation_fairness": 4.0, "equity_offer": 3.5}
    """
    total_score = 0.0
    details = {}
    return total_score, details


def score_business(market: Dict) -> Tuple[float, Dict[str, float]]:
    """
    비즈니스 경제성 점수를 계산한다. (하드코딩)

    Parameters:
        market (dict): 비즈니스 관련 지표 포함. 예: {
            "ltv_usd": 1200,
            "cac_usd": 300,
            "gross_margin_pct": 65,
            "revenue_model_clarity": "high"
        }

    Returns:
        total_score (float): 0~5점
        details (dict): {"ltv_cac": 5.0, "gross_margin": 4.0, "revenue_model": 4.5}
    """
    total_score = 0.0
    details = {}
    return total_score, details

In [ ]:
def scoring_node(state: GraphState) -> GraphState:
    """
    모든 카테고리별 스코어링 함수를 호출하고 state에 점수 정보를 저장한다.
    """

    # 1️⃣ 개별 점수 계산
    founder_score, founder_details = score_founder(state["startup"])
    market_score, market_details = score_market(state["market"])
    technology_score, tech_details = score_technology(state["technology"])
    moat_score, moat_details = score_moat(state["competition"])
    traction_score, traction_details = score_traction(state["market"])
    terms_score, terms_details = score_terms(state["deal_terms"])
    business_score, business_details = score_business(state["market"])

    # 2️⃣ 카테고리별 점수 합산
    scores = {
        "founder": founder_score,
        "market": market_score,
        "technology": technology_score,
        "moat": moat_score,
        "traction": traction_score,
        "terms": terms_score,
        "business": business_score,
    }

    # 3️⃣ 세부 점수 저장
    score_details = {
        "founder": founder_details,
        "market": market_details,
        "technology": tech_details,
        "moat": moat_details,
        "traction": traction_details,
        "terms": terms_details,
        "business": business_details,
    }

    # 4️⃣ state에 저장
    state["scores"] = scores
    state["score_details"] = score_details

    return state

In [ ]:
def aggregate_scores(scores: Dict[str, float]) -> float:
    """
    카테고리별 점수를 가중합해 0~100 기준 점수(base_score)로 환산한다.
    """
    weights = {
        "founder": 0.10,
        "market": 0.20,
        "technology": 0.20,
        "moat": 0.15,
        "traction": 0.15,
        "terms": 0.05,
        "business": 0.10,
    }

    total = sum(scores.get(k, 0) * 20 * w for k, w in weights.items())
    return round(total, 2)